# 2. Spam

- Reportar el porcentaje de correos etiquetados como spam y no spam en el conjunto de datos.
- Dividir aleatoriamente el conjunto de datos en 70% entrenamiento y 30% restante como validacion. Usar 0 como semilla para el generador de numeros aleatorios.
- Entrenar 2 clasificadores bayesianos ingenuos con distintas distribuciones.
- Emplear los clasificadores entrenados para predecir spam tanto en los datos de entrenamiento como en los de validacion y reportar el porcentaje de predicciones correctas de cada clasificador
- Discutir el desempeno de los diferentes clasificadores

## Parte (1/2) : Clasificador condistribuciones de Bernoulli a patita de perro muerto (manual)

@Flores-Silva P.

In [1]:
import pandas as pd
import numpy as np

spam = pd.read_csv('spam.csv',header = None,sep = ' ')
Spam = pd.DataFrame(spam)
numero_filas = Spam.shape[0]
numero_columnas = Spam.shape[1]
print('El numero de filas es %d' %numero_filas + '. El numero de columnas es %d' %numero_columnas + '.')

#Renombramos la ultima columna del dataframe
Spam.rename(columns={2000:'Spam?'}, inplace=True)

#imprimimos el dataframe
Spam.info()
Spam.head(10)

El numero de filas es 5172. El numero de columnas es 2001.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5172 entries, 0 to 5171
Columns: 2001 entries, 0 to Spam?
dtypes: int64(2001)
memory usage: 79.0 MB


,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,Spam?
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,3,0,0,0,0,0,3,0,0,7,...,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,3,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [2]:
#Sobre la columna Spam? buscamos los valores que contengan el string 0,
#es decir que no sean spam
no_spam = Spam.loc[Spam['Spam?'] == 0]
#Hacemos otro data frame que contenga los datos de los correos que no fueron spam
#con la misma llave que el dataframe original, esto se hace por si nos llegara a servir
No_Spam = pd.DataFrame(no_spam)
No_Spam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3672 entries, 1050 to 5171
Columns: 2001 entries, 0 to Spam?
dtypes: int64(2001)
memory usage: 56.1 MB


In [3]:
#Hacemos lo mismo para el correo clasificado como spam
#es decir que los valores de la columna Spam? sean 1
es_spam = Spam.loc[Spam['Spam?'] == 1]
#Hacemos otro data frame que contenga los datos de los correos que no fueron spam
#con la misma llave que el dataframe original, esto se hace por si nos llegara a servir
Es_Spam = pd.DataFrame(es_spam)
Es_Spam.info()
print('\n')
#reportamos el numero de correos spam y no spam
print('En este DataFrame hay %d' %es_spam.shape[0] + ' correos spam y hay %d' %no_spam.shape[0] + ' correos buenos')
print('Es decir, el %.2f porciento es spam, ' %(es_spam.shape[0]*100/Spam.shape[0]) + 'mientras que el %.2f es correo bueno' %(no_spam.shape[0]*100/Spam.shape[0]))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1500 entries, 0 to 4069
Columns: 2001 entries, 0 to Spam?
dtypes: int64(2001)
memory usage: 22.9 MB


En este DataFrame hay 1500 correos spam y hay 3672 correos buenos
Es decir, el 29.00 porciento es spam, mientras que el 71.00 es correo bueno


In [4]:
#Definimos nuestros conjuntos de entrenamiento y validacion
#train se construye a partir de la funcion sample() donde frac es el porcentaje de los datos
#random_state es la semilla del numero aleatorio, esta funcion por default tiene desactivado
#el ingreso de datos por reemplazo, es decir no se pueden repetir filas
#el conjunto test se construye a partir del train quitando todos los indices que tiene train,
#es decir hace una operacion de conjuntos del conjunto universo Spam resta el conjunto train 
#y guarda lo que resta en test
train = Spam.sample(frac=0.7,random_state=0)
test = Spam.drop(train.index)
#reseteamos los indices con la funcion reset_index. drop=True lo usamos para evitar que se
#anada una columna del indice anterior
train.reset_index(drop=True,inplace=True)

Train = pd.DataFrame(train)

#Imprimimos nuestro conjunto de entrenamiento
Train

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,Spam?
0,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3616,0,0,0,0,0,2,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
3617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3618,0,0,0,0,2,0,0,0,0,9,...,0,0,0,0,0,0,0,0,0,1


In [5]:
#verificamos la cantidad de spam y no spam en los datos de entrenamiento
spam_en_train = train.loc[train['Spam?'] == 1]
nospam_en_train = train.loc[train['Spam?'] == 0]

print('En los datos de entrenamiento hay %d spam' %spam_en_train.shape[0] + ' y hay %d correo buenos' %nospam_en_train.shape[0])

#reseteamos los indices de ambos
spam_en_train.reset_index(drop=True,inplace=True)
nospam_en_train.reset_index(drop=True,inplace=True)
#Para evitar errores de copia fue necesario crear un dataframe nuevo a partir de los anteriores
#note que estos nuevos DF comienzan con mayuscula
Spam_en_train = pd.DataFrame(spam_en_train)
Nospam_en_train = pd.DataFrame(nospam_en_train)

En los datos de entrenamiento hay 1069 spam y hay 2551 correo buenos


In [6]:
#reseteamos los indices con la funcion reset_index. drop=True lo usamos para evitar que se
#anada una columna del indice anterior
test.reset_index(drop=True,inplace=True)
Test = pd.DataFrame(test)
#imprimimos nuestro conjunto de verificacion
Test

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,Spam?
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,0,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
1548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1549,0,0,0,0,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#verificamos la cantidad de spam y no spam en los datos de verificacion
spam_en_test = test.loc[test['Spam?'] == 1]
nospam_en_test = test.loc[test['Spam?'] == 0]

print('En los datos de test hay %d spam' %spam_en_test.shape[0] + ' y hay %d correo buenos' %nospam_en_test.shape[0])

#reseteamos los indices de ambos
spam_en_test.reset_index(drop=True,inplace=True)
nospam_en_test.reset_index(drop=True,inplace=True)
#Para evitar errores de copia fue necesario crear un dataframe nuevo a partir de los anteriores
#note que estos nuevos DF comienzan con mayuscula
Spam_en_test = pd.DataFrame(spam_en_train)
Nospam_en_test = pd.DataFrame(nospam_en_train)

En los datos de test hay 431 spam y hay 1121 correo buenos


### Clasificador Bayesiano ingenuo: Distribución de Bernoulli

Consideramos que cada palabra sigue una distribución de Bernoulli, es decir, aparece o no. A partir de ello entremas un clasificador bayesiano con MAP.

Tenemos que, 
<br>
$$\displaystyle{P(Correo|Clase)=\prod\limits_{i=1}^{|pal|}Be(w_t;q) = \prod\limits_{i=1}^{|pal|}q^{w_t}(1-q)^{1-w_t}}$$
<br>
donde el estimdor a maximo posteriori es
<br>
$$\hat{q}_{MAP(w_t|C)}=\frac{\sum\limits_{i=1}^{n}x_i+\alpha-1}{n_c+\beta+\alpha-2}$$
<br>
$$\hat{q}_{MAP(C)}=\frac{n_c+\alpha-1}{n+\beta+\alpha-2}$$

donde $|pal|$ es la cantidad de palabras (2000); $|\alpha|=2$; $\beta=|pal|$ para el parametro de los atributos y $\beta=2$ para el parametro de la clase; $n_c$ numero de correos de la clase C; $n$ numero total de correos
<br>
$\sum\limits_{i=1}^{n}x_i=n_c(w_t)$ numero de correos de la clase C donde aparece $w_t$

In [8]:
#Dado que es bernoulli para todas las palabras, entonces tenemos que poner todos los valores diferentes
#de 0 a 1, esto se hace con la siguiente instruccion, esto no afecta el valor de spam ya que solo hay 1 o 0
#pero como es necesario trabajar solo con spam o no spam usaremos los dataframes spam_en_train y nospam_en_train
#definidos anteriormente
Spam_en_train[Spam_en_train != 0] = 1

#Quitamos la columna "Spam?" ya que no nos sera util ya que sabemos que estos correos son spam
Spam_en_train.drop("Spam?",axis=1,inplace=True)

#imprimimos train
Spam_en_train

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1066,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1067,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
#Hacemos lo mismo para test
Nospam_en_train[Nospam_en_train != 0] = 1

#Quitamos la columna "Spam?" ya que no nos sera util ya que sabemos que estos correos son spam
Nospam_en_train.drop("Spam?",axis=1,inplace=True)

#imprimimos test
Nospam_en_train

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2546,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2547,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2548,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2549,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Ahora calculamos los parametros para la clase Spam $$\hat{q}_{MAP(w_t|Spam)}=\frac{\sum\limits_{i=1}^{n}x_i+\alpha-1}{n_c+\beta+\alpha-2}$$
<br>
Para ello hacemos una matriz transpuesta de puros 1 y la multiplicamos por la transpuesta de las matrices Spam no Spam
por definicion este producto nos va a resultar en un vector de 1x1 donde cada entrada representa la suma o el numero de apariciones de la palabra en todos los correos ya que las filas de las matrices transpuestas son las palabras, mientras que las columnas seran los correos

In [10]:
#Matriz de (1069x1)
unos = np.ones((1069,1),dtype=int)
#Matriz Transpuesta del Spam entrenamiento
matriz_spam_train = Spam_en_train.T.to_numpy()
#producto matricial, la ultima fila corresponde al total de correos spam
#este valor no nos va a interesar
producto_spam = matriz_spam_train @ unos
print(producto_spam.shape)
producto_spam

(2000, 1)


array([[344],
       [ 22],
       [  9],
       ...,
       [  5],
       [ 11],
       [  9]], dtype=int64)

Definimos $\alpha=2$, $\beta=2000$ numero de palabras, $n_c=1069$ numero de correos spam.
De esta forma la constante es $$\frac{1}{2000+1069}=\frac{1}{3069}$$
<br>
al vector producto hay que dividirlo entre 3069 y sumarle la constante, esto se hace a continuacion

In [11]:
#multiplicamos todas las entradas por 1/3069
producto_spam = producto_spam/3069
print(producto_spam)

[[0.11208863]
 [0.00716846]
 [0.00293255]
 ...
 [0.0016292 ]
 [0.00358423]
 [0.00293255]]


In [12]:
#sumamos todas las entradas con 1/3069
producto_spam = producto_spam+ 1/3069
print(producto_spam)

[[0.11241447]
 [0.0074943 ]
 [0.00325839]
 ...
 [0.00195503]
 [0.00391007]
 [0.00325839]]


Ahora calculamos no q, recordemos que, hasta este punto, las entradas de producto_spam contienen todas las q. No q se calcula con una matriz de unos y la resta con producto_spam

In [13]:
no_producto_spam = np.ones((1069,1),dtype=int)
no_producto_spam = 1 - producto_spam

#corroboremos que la suma si da uno
print(no_producto_spam + producto_spam)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


Ahora Calculamos $$\hat{q}_{MAP(Spam)}=\frac{n_c+\alpha-1}{n+\beta+\alpha-2}$$, donde $n$ es el numero de correos totales = 5172 para este caso $\alpha=\beta=2$, Asi $q=\frac{1069+1}{5172+2}=\frac{535}{2587}$

In [14]:
q_spam = 535/2587
q_spam

0.20680324700425204

Ahora calculamos los parametros para la clase No_spam $$\hat{q}_{MAP(w_t|No_spam)}=\frac{\sum\limits_{i=1}^{n}x_i+\alpha-1}{n_c+\beta+\alpha-2}$$
<br>
Para ello hacemos una matriz transpuesta de puros 1 y la multiplicamos por la transpuesta de las matrices Spam no Spam
por definicion este producto nos va a resultar en un vector de 1x1 donde cada entrada representa la suma o el numero de apariciones de la palabra en todos los correos ya que las filas de las matrices transpuestas son las palabras, mientras que las columnas seran los correos

In [15]:
#Matriz de (2551x1)
unos_nospam = np.ones((2551,1),dtype=int)
#Matriz Transpuesta del no_spam entrenamiento
matriz_no_spam_train = Nospam_en_train.T.to_numpy()

#producto matricial, la ultima fila corresponde al total de correos spam
#este valor no nos va a interesar
producto_no_spam = matriz_no_spam_train @ unos_nospam
print(producto_no_spam)
producto_no_spam.shape

[[98]
 [ 1]
 [ 0]
 ...
 [ 1]
 [ 2]
 [ 1]]


(2000, 1)

Definimos $\alpha=2$, $\beta=2000$ numero de palabras, $n_c=2551$ numero de correos no_spam.
De esta forma la constante es $$\frac{1}{2000+2551}=\frac{1}{4551}$$
<br>
al vector producto hay que dividirlo entre 4551 y sumarle la constante, esto se hace a continuacion

In [16]:
#multiplicamos todas las entradas por 1/4551
producto_no_spam = producto_no_spam/4551
print(producto_no_spam)

[[0.02153373]
 [0.00021973]
 [0.        ]
 ...
 [0.00021973]
 [0.00043946]
 [0.00021973]]


In [17]:
#sumamos todas las entradas con 1/4551
producto_no_spam = producto_no_spam+ 1/4551
print(producto_no_spam)

[[0.02175346]
 [0.00043946]
 [0.00021973]
 ...
 [0.00043946]
 [0.0006592 ]
 [0.00043946]]


Ahora calculamos no q, recordemos que, hasta este punto, las entradas de producto_spam contienen todas las q. No q se calcula con una matriz de unos y la resta con producto_spam

In [18]:
no_producto_no_spam = np.ones((2551,1),dtype=int)
no_producto_no_spam = 1 - producto_no_spam

#corroboremos que la suma si da uno
print(no_producto_no_spam + producto_no_spam)

[[1.]
 [1.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


Ahora Calculamos $$\hat{q}_{MAP(no_spam)}=\frac{n_c+\alpha-1}{n+\beta+\alpha-2}$$, donde $n$ es el numero de correos totales = 5172 para este caso $\alpha=\beta=2$, Asi $q=\frac{2551+1}{5172+2}=\frac{1276}{2587}$

In [19]:
q_no_spam = 1276/2587
q_no_spam

0.49323540780827213

### Aqui Termina el entrenamiento 

### Clasificacion

In [20]:
#nuestros datos test tienen que ser "renormalizados" de acuerdo a la distribucion de Bernoulli
#Tal y como se hizo con los datos de entrenamiento, es decir cambiar los valores mayores o iguales
#a 1 por 1. Esto se hace a continuacion.
Test[Test != 0] = 1

#Imprimimos los datos de prueba
Test

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,Spam?
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1549,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
#Transformamos los datos a una matriz, sin la ultima columna
matriz_test = Test.drop("Spam?",axis=1).to_numpy()
print(matriz_test)
print(matriz_test.shape)

#Creamos una matriz auxiliar que contendra las probabilidades segun la entrada y es para la clase spam, el tamano es de 
#1552 ya que tenemos 1552 datos a validar.
matriz_spam = np.zeros((1552,2000),dtype=float)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(1552, 2000)


Para Clasificar lo que se va a hacer es recorrer todos los datos con dos ciclos for. Uno de ellos servira para iterar sobre cada correo, mientras que el otro for sirve para iterar sobre cada palabra (0 a 1999). Esta iteracion se hace para decidir si se usa la probabilidad o la (1-p). Para el caso en el que se explora la clase Spam si en la entrada se ve un 1 se usa la probabilidad del vector producto spam contenida en la posicion en el cual aparecio el 1, ya que recordemos por construccion las posiciones del vector corresponden a una determinada palabra. Si en la entrada ve un 0 usamos la probabilidad del vector no_producto_spam contenido en la posicion en el cual aparecio el 0, las posiciones son marcadas por la letra j y van de 0 a 1999. Cada probabilidad segun la entrada se guarda en la matriz_spam en pocas palabras lo que estamos haciendo es poner las probabilidades calculadas de cada valor en la tabla en una matriz de las mismas dimensiones

In [22]:
for i in range (matriz_test.shape[0]):
    for j in range (matriz_test.shape[1]):
        if matriz_test[i][j] == 0:
            matriz_spam[i][j] = no_producto_spam[j]
        else:
            matriz_spam[i][j] = producto_spam[j]

In [23]:
#Imprimimos la matriz recien modificada
matriz_spam

array([[0.88758553, 0.9925057 , 0.99674161, ..., 0.99804497, 0.99608993,
        0.99674161],
       [0.88758553, 0.9925057 , 0.99674161, ..., 0.99804497, 0.99608993,
        0.99674161],
       [0.88758553, 0.9925057 , 0.99674161, ..., 0.99804497, 0.99608993,
        0.99674161],
       ...,
       [0.88758553, 0.9925057 , 0.99674161, ..., 0.99804497, 0.99608993,
        0.99674161],
       [0.88758553, 0.9925057 , 0.99674161, ..., 0.99804497, 0.99608993,
        0.99674161],
       [0.88758553, 0.9925057 , 0.99674161, ..., 0.99804497, 0.99608993,
        0.99674161]])

In [24]:
#Creamos otra matriz auxiliar para la clase no spam
matriz_nospam = np.zeros((1552,2000),dtype=float)

In [25]:
#rellenamos la matriz auxiliar con el procedimiento ya explicado
for i in range (matriz_test.shape[0]):
    for j in range (matriz_test.shape[1]):
        if matriz_test[i][j] == 0:
            matriz_nospam[i][j] = no_producto_no_spam[j]
        else:
            matriz_nospam[i][j] = producto_no_spam[j]

In [26]:
#Imprimimos la matriz recien modificada
matriz_nospam

array([[0.97824654, 0.99956054, 0.99978027, ..., 0.99956054, 0.9993408 ,
        0.99956054],
       [0.97824654, 0.99956054, 0.99978027, ..., 0.99956054, 0.9993408 ,
        0.99956054],
       [0.97824654, 0.99956054, 0.99978027, ..., 0.99956054, 0.9993408 ,
        0.99956054],
       ...,
       [0.97824654, 0.99956054, 0.99978027, ..., 0.99956054, 0.9993408 ,
        0.99956054],
       [0.97824654, 0.99956054, 0.99978027, ..., 0.99956054, 0.9993408 ,
        0.99956054],
       [0.97824654, 0.99956054, 0.99978027, ..., 0.99956054, 0.9993408 ,
        0.99956054]])

Una vez que tenemos las probabilidades de las dos clases en dos matrices separadas. Por construccion hay que calcular el producto de todas las probabilidades de cada palabra para cada correo, por ello es que en realidad se usan las matrices.
Para ello definimos un vector de dimension 1552 donde cada entrada contiene el producto de todas las palabras. Esto ultimo se hace con la funcion prod(matriz,axis=1).

In [27]:
#vector de probabilidades de los datos de prueba fijados para la clase spam
probabilidades_spam = np.prod(matriz_spam,axis=1)
print(probabilidades_spam)
print(len(probabilidades_spam))

[4.99777769e-081 5.54624646e-013 2.35869347e-006 ... 3.33292506e-147
 4.57237114e-037 1.00945007e-025]
1552


In [28]:
#vector de probabilidades de los datos de prueba fijados para la clase no spam
probabilidades_nospam = np.prod(matriz_nospam,axis=1)
print(probabilidades_nospam)
print(len(probabilidades_nospam))

[1.04338116e-101 1.16646829e-015 9.46109275e-007 ... 7.46014988e-093
 2.71077477e-028 1.66844408e-014]
1552


In [29]:
#Multiplicamos por las probabilidades de clase ya calculadas previamente
probabilidades_spam = probabilidades_spam * q_spam
print(probabilidades_spam)

[1.03355665e-081 1.14698178e-013 4.87785467e-007 ... 6.89259725e-148
 9.45581198e-038 2.08757553e-026]


In [30]:
#Multiplicamos por las probabilidades de clase ya calculadas previamente
probabilidades_nospam = probabilidades_nospam * q_no_spam
print(probabilidades_nospam)

[5.14632534e-102 5.75343464e-016 4.66654594e-007 ... 3.67961007e-093
 1.33705010e-028 8.22935697e-015]


Una vez calculados los productos basta con elegir el valor maximo de entre las dos clases. Para ello creamos un array llamado estimacion. Recorremos ambos arrays y decidimos cual es el valor mas grande y escribimos 1 o 0 dependiendo el valor mas grande en la entrada i del vector estimacion. Si hay valores que sean iguales, entonces escribimos -1

In [31]:
#Creamos vector estimacion
Estimacion = np.zeros((1552,1),dtype=int)

#iteramos sobre los vectores de probabilidades de clase
for i in range(1552):
    if probabilidades_nospam[i]>probabilidades_spam[i]: #No es spam
        Estimacion[i] = 0
    elif probabilidades_nospam[i]<probabilidades_spam[i]: ##Es spam
        Estimacion[i] = 1
    else: ##No sabemos que es
        Estimacion[i] = -1

In [32]:
#imprimimos el vector para verifivar los casos donde se escribio -1
for i in range(Estimacion.shape[0]):
    print (i,Estimacion[i])

0 [1]
1 [1]
2 [1]
3 [1]
4 [1]
5 [1]
6 [1]
7 [1]
8 [1]
9 [1]
10 [1]
11 [1]
12 [1]
13 [1]
14 [1]
15 [1]
16 [1]
17 [1]
18 [1]
19 [1]
20 [1]
21 [1]
22 [1]
23 [1]
24 [1]
25 [1]
26 [1]
27 [1]
28 [1]
29 [1]
30 [1]
31 [1]
32 [0]
33 [1]
34 [1]
35 [1]
36 [1]
37 [1]
38 [1]
39 [1]
40 [1]
41 [1]
42 [1]
43 [1]
44 [1]
45 [1]
46 [1]
47 [1]
48 [1]
49 [1]
50 [1]
51 [1]
52 [1]
53 [1]
54 [1]
55 [1]
56 [1]
57 [1]
58 [1]
59 [1]
60 [1]
61 [1]
62 [1]
63 [1]
64 [1]
65 [1]
66 [1]
67 [1]
68 [1]
69 [1]
70 [1]
71 [-1]
72 [1]
73 [1]
74 [1]
75 [1]
76 [1]
77 [1]
78 [1]
79 [1]
80 [1]
81 [1]
82 [1]
83 [1]
84 [1]
85 [1]
86 [1]
87 [1]
88 [1]
89 [1]
90 [1]
91 [1]
92 [1]
93 [-1]
94 [1]
95 [1]
96 [-1]
97 [1]
98 [1]
99 [1]
100 [1]
101 [1]
102 [1]
103 [1]
104 [1]
105 [1]
106 [1]
107 [1]
108 [1]
109 [1]
110 [1]
111 [1]
112 [1]
113 [1]
114 [1]
115 [1]
116 [-1]
117 [1]
118 [1]
119 [1]
120 [1]
121 [1]
122 [1]
123 [1]
124 [1]
125 [1]
126 [1]
127 [1]
128 [1]
129 [1]
130 [1]
131 [1]
132 [1]
133 [1]
134 [1]
135 [1]
136 [1]
137 [1]
13

1085 [0]
1086 [1]
1087 [1]
1088 [1]
1089 [1]
1090 [1]
1091 [1]
1092 [1]
1093 [1]
1094 [1]
1095 [1]
1096 [1]
1097 [1]
1098 [1]
1099 [1]
1100 [1]
1101 [1]
1102 [1]
1103 [-1]
1104 [1]
1105 [1]
1106 [1]
1107 [1]
1108 [1]
1109 [1]
1110 [1]
1111 [1]
1112 [1]
1113 [1]
1114 [1]
1115 [1]
1116 [1]
1117 [1]
1118 [1]
1119 [1]
1120 [1]
1121 [1]
1122 [1]
1123 [1]
1124 [1]
1125 [1]
1126 [1]
1127 [1]
1128 [1]
1129 [1]
1130 [1]
1131 [1]
1132 [1]
1133 [1]
1134 [1]
1135 [1]
1136 [1]
1137 [1]
1138 [1]
1139 [1]
1140 [1]
1141 [1]
1142 [1]
1143 [1]
1144 [1]
1145 [1]
1146 [1]
1147 [1]
1148 [1]
1149 [1]
1150 [1]
1151 [1]
1152 [1]
1153 [1]
1154 [1]
1155 [1]
1156 [1]
1157 [1]
1158 [1]
1159 [1]
1160 [1]
1161 [1]
1162 [1]
1163 [1]
1164 [1]
1165 [1]
1166 [1]
1167 [1]
1168 [1]
1169 [1]
1170 [1]
1171 [1]
1172 [1]
1173 [1]
1174 [1]
1175 [1]
1176 [1]
1177 [1]
1178 [1]
1179 [1]
1180 [1]
1181 [1]
1182 [1]
1183 [1]
1184 [1]
1185 [1]
1186 [1]
1187 [1]
1188 [1]
1189 [1]
1190 [1]
1191 [1]
1192 [1]
1193 [1]
1194 [1]
1195 [-1]

Observamos que estos casos raros tienen valor 0.0, quiza se deba a una inestabilidad numerica.

In [33]:
print(probabilidades_nospam[71])
print(probabilidades_spam[71])

0.0
0.0


Anadimos una columna estimacion a nuestra tabla original Test, los valores de esta columna corresponden a los valores del vector estimacion

In [34]:
Test['Prediccion'] = Estimacion
Test

,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,Spam?,Prediccion
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1549,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Anadimos otra nueva columna llamada comparacion que, tendra valor cero si los valores de la columna 'Spam?' y 'Prediccion'
son los mismos y tendra valor 1 si los valores son diferentes. Esto nos permitira conocer cuantas predicciones malas se hicieron.

In [35]:
#anade columna
Test['Comparacion'] = np.where(Test['Spam?'] != Test['Prediccion'], 1, 0)

In [36]:
#Imprime
Test

,0,1,2,3,4,5,6,7,8,9,...,1993,1994,1995,1996,1997,1998,1999,Spam?,Prediccion,Comparacion
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1548,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1549,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1550,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
#Dado que en la columna comparacion solo hay 0 o 1 sumamos los valores en toda lka columna mediante la funcion sum()
Errores = Test['Comparacion'].sum()
print('Hay %d muestras mal clasificadas en el dataset de validacion.' %Errores + ' Corresponde al %.2f porciento de error' %(Errores*100/Test.shape[0]))

Hay 66 muestras mal clasificadas en el dataset de validacion. Corresponde al 4.25 porciento de error


La clasificacion con los datos de entrenamiento se hace a continuacion. Los datos de entrenamiento sin separar se encuentran en el data frame Train definido anteriormente. Se procede igual que en el caso anterior 

In [38]:
#nuestros datos test tienen que ser "renormalizados" de acuerdo a la distribucion de Bernoulli
#Tal y como se hizo con los datos de entrenamiento, es decir cambiar los valores mayores o iguales
#a 1 por 1. Esto se hace a continuacion.
Train[Train != 0] = 1

#Imprimimos los datos de prueba
Train

,0,1,2,3,4,5,6,7,8,9,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,Spam?
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3616,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3618,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


In [39]:
#Transformamos los datos a una matriz, sin la ultima columna
matriz_train = Train.drop("Spam?",axis=1).to_numpy()
print(matriz_train)
print(matriz_train.shape)

#Creamos una matriz auxiliar que contendra las probabilidades segun la entrada y es para la clase spam, el tamano es de 
#1552 ya que tenemos 1552 datos a validar.
matriz_spam1 = np.zeros((3620,2000),dtype=float)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
(3620, 2000)


In [40]:
#ciclo para rellenar la matriz spam1
for i in range (matriz_train.shape[0]):
    for j in range (matriz_train.shape[1]):
        if matriz_train[i][j] == 0:
            matriz_spam1[i][j] = no_producto_spam[j]
        else:
            matriz_spam1[i][j] = producto_spam[j]

In [41]:
#Creamos otra matriz auxiliar para la clase no spam
matriz_nospam1 = np.zeros((3620,2000),dtype=float)

In [42]:
#rellenamos la matriz auxiliar con el procedimiento ya explicado
for i in range (matriz_train.shape[0]):
    for j in range (matriz_train.shape[1]):
        if matriz_train[i][j] == 0:
            matriz_nospam1[i][j] = no_producto_no_spam[j]
        else:
            matriz_nospam1[i][j] = producto_no_spam[j]

In [43]:
#vector de probabilidades de los datos de prueba fijados para la clase spam
probabilidades_spam1 = np.prod(matriz_spam1,axis=1)
print(probabilidades_spam1)
print(len(probabilidades_spam1))

[3.44849878e-065 1.09690301e-013 4.59900968e-051 ... 1.98751203e-077
 5.50427852e-280 7.59737219e-082]
3620


In [44]:
#vector de probabilidades de los datos de prueba fijados para la clase no spam
probabilidades_nospam1 = np.prod(matriz_nospam1,axis=1)
print(probabilidades_nospam1)
print(len(probabilidades_nospam1))

[1.01590461e-42 4.19116366e-15 1.34634690e-31 ... 2.44524413e-54
 0.00000000e+00 1.22960134e-72]
3620


In [45]:
#Multiplicamos por las probabilidades de clase ya calculadas previamente
probabilidades_spam1 = probabilidades_spam1 * q_spam
print(probabilidades_spam1)

#Multiplicamos por las probabilidades de clase ya calculadas previamente
probabilidades_nospam1 = probabilidades_nospam1 * q_no_spam
print(probabilidades_nospam1)

[7.13160745e-066 2.26843104e-014 9.51090135e-052 ... 4.11023942e-078
 1.13830267e-280 1.57116124e-082]
[5.01080126e-43 2.06723032e-15 6.64065963e-32 ... 1.20608098e-54
 0.00000000e+00 6.06482917e-73]


In [46]:
#Creamos vector estimacion
Estimacion1 = np.zeros((3620,1),dtype=int)

#iteramos sobre los vectores de probabilidades de clase
for i in range(1552):
    if probabilidades_nospam1[i]>probabilidades_spam1[i]: #No es spam
        Estimacion1[i] = 0
    elif probabilidades_nospam1[i]<probabilidades_spam1[i]: ##Es spam
        Estimacion1[i] = 1
    else: ##No sabemos que es
        Estimacion1[i] = -1

In [47]:
for i in range(Estimacion1.shape[0]):
    print (i,Estimacion1[i])

0 [0]
1 [1]
2 [0]
3 [1]
4 [0]
5 [1]
6 [0]
7 [0]
8 [1]
9 [0]
10 [0]
11 [1]
12 [1]
13 [0]
14 [0]
15 [0]
16 [0]
17 [1]
18 [1]
19 [0]
20 [0]
21 [0]
22 [0]
23 [1]
24 [0]
25 [0]
26 [0]
27 [1]
28 [0]
29 [1]
30 [1]
31 [0]
32 [0]
33 [0]
34 [0]
35 [0]
36 [1]
37 [0]
38 [0]
39 [1]
40 [1]
41 [0]
42 [0]
43 [0]
44 [1]
45 [1]
46 [1]
47 [0]
48 [0]
49 [0]
50 [1]
51 [1]
52 [0]
53 [0]
54 [0]
55 [1]
56 [-1]
57 [0]
58 [0]
59 [0]
60 [0]
61 [0]
62 [1]
63 [0]
64 [0]
65 [0]
66 [0]
67 [1]
68 [0]
69 [0]
70 [1]
71 [1]
72 [-1]
73 [0]
74 [0]
75 [1]
76 [0]
77 [1]
78 [1]
79 [0]
80 [1]
81 [1]
82 [0]
83 [0]
84 [0]
85 [1]
86 [0]
87 [1]
88 [0]
89 [0]
90 [0]
91 [1]
92 [0]
93 [0]
94 [0]
95 [0]
96 [0]
97 [0]
98 [1]
99 [1]
100 [1]
101 [1]
102 [1]
103 [0]
104 [0]
105 [0]
106 [0]
107 [1]
108 [0]
109 [1]
110 [1]
111 [0]
112 [0]
113 [0]
114 [0]
115 [0]
116 [1]
117 [0]
118 [1]
119 [0]
120 [1]
121 [1]
122 [0]
123 [1]
124 [0]
125 [1]
126 [0]
127 [0]
128 [1]
129 [0]
130 [0]
131 [0]
132 [0]
133 [1]
134 [0]
135 [0]
136 [0]
137 [0]
138 

1245 [0]
1246 [0]
1247 [0]
1248 [1]
1249 [0]
1250 [0]
1251 [0]
1252 [0]
1253 [0]
1254 [1]
1255 [0]
1256 [0]
1257 [0]
1258 [0]
1259 [1]
1260 [0]
1261 [1]
1262 [0]
1263 [0]
1264 [0]
1265 [0]
1266 [0]
1267 [0]
1268 [1]
1269 [0]
1270 [1]
1271 [1]
1272 [0]
1273 [1]
1274 [0]
1275 [0]
1276 [1]
1277 [0]
1278 [0]
1279 [0]
1280 [0]
1281 [0]
1282 [1]
1283 [0]
1284 [0]
1285 [1]
1286 [1]
1287 [0]
1288 [0]
1289 [0]
1290 [0]
1291 [0]
1292 [0]
1293 [0]
1294 [1]
1295 [1]
1296 [0]
1297 [1]
1298 [0]
1299 [0]
1300 [0]
1301 [0]
1302 [0]
1303 [0]
1304 [0]
1305 [-1]
1306 [1]
1307 [1]
1308 [0]
1309 [0]
1310 [1]
1311 [0]
1312 [1]
1313 [0]
1314 [0]
1315 [0]
1316 [0]
1317 [0]
1318 [0]
1319 [0]
1320 [0]
1321 [1]
1322 [0]
1323 [1]
1324 [0]
1325 [1]
1326 [1]
1327 [0]
1328 [0]
1329 [1]
1330 [1]
1331 [0]
1332 [0]
1333 [1]
1334 [0]
1335 [0]
1336 [0]
1337 [0]
1338 [0]
1339 [0]
1340 [1]
1341 [1]
1342 [0]
1343 [0]
1344 [0]
1345 [0]
1346 [1]
1347 [1]
1348 [1]
1349 [0]
1350 [0]
1351 [0]
1352 [0]
1353 [0]
1354 [1]
1355 [0]


2220 [0]
2221 [0]
2222 [0]
2223 [0]
2224 [0]
2225 [0]
2226 [0]
2227 [0]
2228 [0]
2229 [0]
2230 [0]
2231 [0]
2232 [0]
2233 [0]
2234 [0]
2235 [0]
2236 [0]
2237 [0]
2238 [0]
2239 [0]
2240 [0]
2241 [0]
2242 [0]
2243 [0]
2244 [0]
2245 [0]
2246 [0]
2247 [0]
2248 [0]
2249 [0]
2250 [0]
2251 [0]
2252 [0]
2253 [0]
2254 [0]
2255 [0]
2256 [0]
2257 [0]
2258 [0]
2259 [0]
2260 [0]
2261 [0]
2262 [0]
2263 [0]
2264 [0]
2265 [0]
2266 [0]
2267 [0]
2268 [0]
2269 [0]
2270 [0]
2271 [0]
2272 [0]
2273 [0]
2274 [0]
2275 [0]
2276 [0]
2277 [0]
2278 [0]
2279 [0]
2280 [0]
2281 [0]
2282 [0]
2283 [0]
2284 [0]
2285 [0]
2286 [0]
2287 [0]
2288 [0]
2289 [0]
2290 [0]
2291 [0]
2292 [0]
2293 [0]
2294 [0]
2295 [0]
2296 [0]
2297 [0]
2298 [0]
2299 [0]
2300 [0]
2301 [0]
2302 [0]
2303 [0]
2304 [0]
2305 [0]
2306 [0]
2307 [0]
2308 [0]
2309 [0]
2310 [0]
2311 [0]
2312 [0]
2313 [0]
2314 [0]
2315 [0]
2316 [0]
2317 [0]
2318 [0]
2319 [0]
2320 [0]
2321 [0]
2322 [0]
2323 [0]
2324 [0]
2325 [0]
2326 [0]
2327 [0]
2328 [0]
2329 [0]
2330 [0]
2

3321 [0]
3322 [0]
3323 [0]
3324 [0]
3325 [0]
3326 [0]
3327 [0]
3328 [0]
3329 [0]
3330 [0]
3331 [0]
3332 [0]
3333 [0]
3334 [0]
3335 [0]
3336 [0]
3337 [0]
3338 [0]
3339 [0]
3340 [0]
3341 [0]
3342 [0]
3343 [0]
3344 [0]
3345 [0]
3346 [0]
3347 [0]
3348 [0]
3349 [0]
3350 [0]
3351 [0]
3352 [0]
3353 [0]
3354 [0]
3355 [0]
3356 [0]
3357 [0]
3358 [0]
3359 [0]
3360 [0]
3361 [0]
3362 [0]
3363 [0]
3364 [0]
3365 [0]
3366 [0]
3367 [0]
3368 [0]
3369 [0]
3370 [0]
3371 [0]
3372 [0]
3373 [0]
3374 [0]
3375 [0]
3376 [0]
3377 [0]
3378 [0]
3379 [0]
3380 [0]
3381 [0]
3382 [0]
3383 [0]
3384 [0]
3385 [0]
3386 [0]
3387 [0]
3388 [0]
3389 [0]
3390 [0]
3391 [0]
3392 [0]
3393 [0]
3394 [0]
3395 [0]
3396 [0]
3397 [0]
3398 [0]
3399 [0]
3400 [0]
3401 [0]
3402 [0]
3403 [0]
3404 [0]
3405 [0]
3406 [0]
3407 [0]
3408 [0]
3409 [0]
3410 [0]
3411 [0]
3412 [0]
3413 [0]
3414 [0]
3415 [0]
3416 [0]
3417 [0]
3418 [0]
3419 [0]
3420 [0]
3421 [0]
3422 [0]
3423 [0]
3424 [0]
3425 [0]
3426 [0]
3427 [0]
3428 [0]
3429 [0]
3430 [0]
3431 [0]
3

In [48]:
print(probabilidades_nospam1[1133])
print(probabilidades_spam1[1133])

0.0
0.0


In [49]:
Train['Prediccion'] = Estimacion1
Train

,0,1,2,3,4,5,6,7,8,9,...,1992,1993,1994,1995,1996,1997,1998,1999,Spam?,Prediccion
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3616,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3618,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [50]:
#anade columna
Train['Comparacion'] = np.where(Train['Spam?'] != Train['Prediccion'], 1, 0)
Train

,0,1,2,3,4,5,6,7,8,9,...,1993,1994,1995,1996,1997,1998,1999,Spam?,Prediccion,Comparacion
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3615,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3616,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3617,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3618,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1


In [56]:
#Dado que en la columna comparacion solo hay 0 o 1 sumamos los valores en toda lka columna mediante la funcion sum()
Errores1 = Train['Comparacion'].sum()
print('Hay %d muestras mal clasificadas en el data set de Entrenamiento' %Errores1 + ' Corresponde al %.2f porciento de error' %(Errores1*100/Train.shape[0]))

Hay 705 muestras mal clasificadas en el data set de Entrenamiento Corresponde al 19.48 porciento de error


Al comparar los resultados obtenidos para el clasificador bayesiano ingenuo, vemos que al pasar los datos de entrenamiento el clasificador tiene 19.4% de fallas, mientras que para los datos de validacion el clasificador tiene 4.25% de fallas. Estas fallas se deben principalmente a inestabilidades numericas, ya que al observar las posiciones de clasificaciones 'raras' en el vector Estimador y corroborar los valores en las posiciones de los vectores probabilidades_nospam y probabilidades_spam los valores corresponden a 0.0 y no es posible decidir si el correo es spam o no.
Otra Razon por la cual puede suceder esto es porque quiza existan errores en los datos, sin embargo esto es muy improbable.